# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import gmaps
import pandas as pd
import requests 
import numpy as np

# Import API key
from google_api_keys import g_key

In [2]:
# gmaps api key config
gmaps.configure(api_key=g_key)

In [3]:
# import extensions 
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps 

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [4]:
# gmaps test
gmaps_figure = gmaps.figure()
gmaps_figure

Figure(layout=FigureLayout(height='420px'))

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# load the csv exported in Part I to a DataFrame
output_data_file = "city_weather_data.csv"

city_weather_data = pd.read_csv(output_data_file)
city_weather_data = city_weather_data.drop(columns=["Unnamed: 0"])

final_weather_df = city_weather_data.rename(columns={"wind speed": "wind_speed"})
final_weather_df 

,city,country,city id,lat,lon,temp,temp_max,humidity,cloudiness,wind_speed
0,lebu,CL,3883457,-37.62,-73.65,12.64,12.64,75,54,0.23
1,teya,RU,1489656,60.38,92.63,10.76,10.76,74,81,2.38
2,castro,BR,3466704,-24.79,-50.01,15.35,15.35,70,11,2.12
3,richards bay,US,5043181,46.84,-95.91,21.00,21.00,68,1,4.60
4,amla,IN,1278775,21.93,78.12,23.51,23.51,93,85,1.26
...,...,...,...,...,...,...,...,...,...,...
553,wellington,NZ,2179537,-41.29,174.78,7.69,9.00,66,100,4.10
554,maldonado,UY,3441890,-34.67,-54.92,10.00,10.00,61,0,1.50
555,izhma,RU,554830,65.01,53.91,14.66,14.66,68,85,3.48
556,bulawayo,ZW,894701,-20.15,28.58,12.10,12.10,61,28,7.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# use the lat and lon as locations, and humidity as the weight.
locations = final_weather_df[["lat", "lon"]].astype(float)
humidity = final_weather_df["humidity"].astype(float)

In [7]:
# create a humidity heatmap
humidity_heatmap  = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

humidity_heatmap.add_layer(heat_layer)

humidity_heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# create weather_criteria_df
weather_criteria_df = pd.DataFrame(final_weather_df, columns = ["city", "temp_max", "wind_speed", "cloudiness"])

In [52]:
# set weather_criteria_df to narrow down the cities to fit weather conditions.
weather_criteria_df[(weather_criteria_df.cloudiness == 0) 
& (weather_criteria_df.wind_speed < 10) & (weather_criteria_df.temp_max < 80)] 

weather_criteria_df          

,city,temp_max,wind_speed,cloudiness
0,lebu,12.64,0.23,54
1,teya,10.76,2.38,81
2,castro,15.35,2.12,11
3,richards bay,21.00,4.60,1
4,amla,23.51,1.26,85
...,...,...,...,...
553,wellington,9.00,4.10,100
554,maldonado,10.00,1.50,0
555,izhma,14.66,3.48,85
556,bulawayo,12.10,7.49,28


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Store into variable named hotel_df.
hotel_df = pd.DataFrame(city_weather_data, columns=["city", "country", "lat", "lon"])

In [11]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [12]:
# Set parameters to search for hotels with 5000 meters.

params_coordinates = f"{hotel_df['lat'][0]},{hotel_df['lon'][0]}"
params_search = "hotel"
params_radius = 5000
params_type = "hotel"

params = {
    "location": params_coordinates, "keyword": params_search, "radius": params_radius, 
    "type": params_type, "key": g_key
}

# base url
maps_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(maps_url, params=params)

In [13]:
hotel_address = response.json()
hotel_address 

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -37.61034009999999, 'lng': -73.6568911},
    'viewport': {'northeast': {'lat': -37.60899027010727,
      'lng': -73.65554127010726},
     'southwest': {'lat': -37.61168992989272, 'lng': -73.65824092989271}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '51aff33ef8f83c5ce5c13ca310b4da5849b1516f',
   'name': 'Hostal Plaza Lebu',
   'photos': [{'height': 2585,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113412507608741287957">A Google User</a>'],
     'photo_reference': 'CmRaAAAAuhwx6BqEuPIbmmWNyjwJ9a1bkM2ujd-ooXTxNsyA5gRS680CuTIIFl657bpc2_OKDx0jLE_50tn9R_Vjkwjs6ufPnWqrFXqN4fHNJoiQwAmylTXrrFKtZzL-h4bmykSPEhDNlqvJuFiNIEa48hmaY-4LGhTVhkuMNJfMZD9WUI-42VvTeUdTlA',
     'width': 2578}],
   'place_id': 'ChIJI1_UKFTxQZYRqHtqR9NxaMk',
   'plus_code': {'compound_code': '98QV+V6 Villa Los Héroes, Lebu, Chile',
    'gl

In [14]:
hotel_df.iloc[0][0]

'lebu'

In [15]:
# creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from class example
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(maps_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
        
    except IndexError:
        hotel_name.append(np.nan)

In [16]:
hotel_name

['Hostal Plaza Lebu',
 nan,
 'Central Palace Hotel - Castro (PR)',
 'Americas Best Value Inn & Suites - Detroit Lakes MN',
 nan,
 'Casa dos Lagos - AL',
 'Hotel South Coast',
 nan,
 nan,
 'Hotel Grand Chancellor Hobart',
 'Flat Hotel La Benediction',
 'Приватний гуртожиток',
 'Pebbles Inn',
 'Melnicani breathtaking view - Bed & Breakfast',
 'Hotel Paulinho',
 nan,
 'Hotel Origenes',
 'Noxapaga Suites',
 'Hilton Garden Inn Kauai Wailua Bay',
 'Harbour House Hotel',
 'Les Embruns Du Baril',
 'Sømandshjemmet',
 'Gostinitsa Zverevo',
 'Albergo Rita',
 'Royal Amazonia',
 'Four Seasons Hotel Alexandria at San Stefano, Egypt',
 'Green Lodge Holiday Homes',
 'Hotel Monasterio de San Miguel',
 nan,
 'The Gascoyne Hotel',
 'Restaurant Hôtel la Grande Fontaine',
 'Hôtel Hiva Oa Hanakee Lodge',
 'Alchie Guest Hauz',
 "Ellie's Villa",
 'Hotel Gold',
 'Wub Miraf Hotel',
 'InterContinental Bora Bora Resort Thalasso Spa',
 'Grand Naniloa Hotel Hilo - a DoubleTree by Hilton',
 "Land's End Boutique Hote

In [17]:
# drop n/a values
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()

,city,country,lat,lon,Hotel Name
0,lebu,CL,-37.62,-73.65,Hostal Plaza Lebu
2,castro,BR,-24.79,-50.01,Central Palace Hotel - Castro (PR)
3,richards bay,US,46.84,-95.91,Americas Best Value Inn & Suites - Detroit Lak...
5,ponta do sol,PT,38.73,-28.27,Casa dos Lagos - AL
6,selfoss,IS,63.93,-21.00,Hotel South Coast
...,...,...,...,...,...
551,esna,EG,25.29,32.55,Orient Star Dahabyia Isna 05 Nights Each Sunday
552,lugovoy,KZ,42.95,72.76,Altyn Saray
553,wellington,NZ,-41.29,174.78,Wellesley Boutique Hotel
556,bulawayo,ZW,-20.15,28.58,Banff Lodge Hotel
